In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2026-prediction/sample_submission.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/test_input.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/test.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w17.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/output_2023_w05.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/output_2023_w10.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w03.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/output_2023_w18.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w05.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/output_2023_w11.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/output_2023_w12.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/output_2023_w16.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/output_2023_w06.csv
/kaggle/input/nfl-big-data-bowl-2026-prediction/train/input_2023_w18.csv


In [11]:
# 1. Load & Concatenate Input and Output CSV's
base_uri = '/kaggle/input/nfl-big-data-bowl-2026-prediction/train'

input_df = pd.DataFrame()
output_df = pd.DataFrame()

for i in range(1, 19):
    week_code = i if i >= 10 else f'0{i}'
    
    input_csv = pd.read_csv(f'{base_uri}/input_2023_w{week_code}.csv')
    output_csv = pd.read_csv(f'{base_uri}/output_2023_w{week_code}.csv')

    input_df = pd.concat([input_df, input_csv])
    output_df = pd.concat([output_df, output_csv])

print('DataFrames loaded successfully')
print(f'\tInput: {len(input_df)} rows')
print(f'\tOutput: {len(output_df)} rows')

DataFrames loaded successfully
	Input: 4880579 rows
	Output: 562936 rows


In [13]:
# 2. Create Gold Table for Training
def row_id(row):
    return f"{row['game_id']}_{row['play_id']}_{row['nfl_id']}_{row['frame_id']}"

input_df['row_id'] = input_df.apply(row_id, axis=1)
output_df['row_id'] = output_df.apply(row_id, axis=1)

predictable_input_df = input_df[input_df['player_to_predict'] == True].copy()

gold_table = pd.merge(predictable_input_df, output_df, on='row_id')
gold_table.head()

,game_id_x,play_id_x,player_to_predict,nfl_id_x,frame_id_x,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,num_frames_output,ball_land_x,ball_land_y,row_id,game_id_y,play_id_y,nfl_id_y,frame_id_y,x_y,y_y
0,2023090700,101,True,46137,1,right,42,Justin Reid,6-1,204,...,21,63.259998,-0.22,2023090700_101_46137_1,2023090700,101,46137,1,56.22,17.28
1,2023090700,101,True,46137,2,right,42,Justin Reid,6-1,204,...,21,63.259998,-0.22,2023090700_101_46137_2,2023090700,101,46137,2,56.63,16.88
2,2023090700,101,True,46137,3,right,42,Justin Reid,6-1,204,...,21,63.259998,-0.22,2023090700_101_46137_3,2023090700,101,46137,3,57.06,16.46
3,2023090700,101,True,46137,4,right,42,Justin Reid,6-1,204,...,21,63.259998,-0.22,2023090700_101_46137_4,2023090700,101,46137,4,57.48,16.02
4,2023090700,101,True,46137,5,right,42,Justin Reid,6-1,204,...,21,63.259998,-0.22,2023090700_101_46137_5,2023090700,101,46137,5,57.91,15.56


In [19]:
gold_table.columns

Index(['game_id_x', 'play_id_x', 'player_to_predict', 'nfl_id_x', 'frame_id_x',
       'play_direction', 'absolute_yardline_number', 'player_name',
       'player_height', 'player_weight', 'player_birth_date',
       'player_position', 'player_side', 'player_role', 'x_x', 'y_x', 's', 'a',
       'dir', 'o', 'num_frames_output', 'ball_land_x', 'ball_land_y', 'row_id',
       'game_id_y', 'play_id_y', 'nfl_id_y', 'frame_id_y', 'x_y', 'y_y'],
      dtype='object')